In [ ]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Set up the game window
WIDTH = 800
HEIGHT = 600
window = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Global Zombie Survival")

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)

# Countries/Levels data
countries = [
    "United States", "China", "Russia", "United Kingdom", 
    "France", "Germany", "Italy", "Spain", "Jordan"
]
current_country = 0

def show_story_screen(country):
    window.fill((0, 0, 0))
    font = pygame.font.Font(None, 36)
    
    if country == "Jordan":
        story_text = [
            "You've reached Jordan, the final destination.",
            "Here, you share your story with the survivors:",
            f"You cleared {len(countries)-1} countries of zombies,",
            f"collected countless trophies, and saved humanity.",
            "The world will remember your bravery.",
            "",
            "Press SPACE to play again or ESC to quit"
        ]
    else:
        story_text = [
            f"Welcome to {country}!",
            "Clear this country of all zombies",
            "to advance to the next region.",
            "",
            "Press SPACE to begin"
        ]
    
    y_position = HEIGHT // 3
    for line in story_text:
        text = font.render(line, True, WHITE)
        text_rect = text.get_rect(center=(WIDTH//2, y_position))
        window.blit(text, text_rect)
        y_position += 50
    
    pygame.display.flip()
    
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    if country == "Jordan":
                        return "play_again"
                    waiting = False
                if event.key == pygame.K_ESCAPE and country == "Jordan":
                    return "quit"

def game_loop(country):
    # Player settings
    player_size = 50
    player_x = WIDTH // 2 - player_size // 2
    player_y = HEIGHT - player_size - 10
    player_speed = 5
    player_hp = 100
    player_shield = 100
    ammo = 50
    food = 100
    trophies = 0

    # Zombie settings - increase difficulty with each country
    zombie_size = 40
    zombies = []
    zombie_speed = 2 + (current_country * 0.5)
    total_zombies = 50 + (current_country * 10)

    # Collectibles
    collectibles = []
    collectible_types = ['ammo', 'food', 'shield', 'trophy']
    collectible_size = 20

    # Bullet settings
    bullet_size = 5
    bullets = []
    bullet_speed = 7

    # Create initial zombies
    for _ in range(total_zombies):
        zombie = {
            'x': random.randint(0, WIDTH - zombie_size),
            'y': random.randint(-1000, 0),
            'health': 100,
            'rect': pygame.Rect(0, 0, zombie_size, zombie_size)
        }
        zombies.append(zombie)

    # Create initial collectibles
    def spawn_collectible():
        collectible = {
            'type': random.choice(collectible_types),
            'x': random.randint(0, WIDTH - collectible_size),
            'y': random.randint(0, HEIGHT - collectible_size),
            'rect': pygame.Rect(0, 0, collectible_size, collectible_size)
        }
        collectibles.append(collectible)

    for _ in range(10):
        spawn_collectible()

    def draw_player(x, y):
        pygame.draw.rect(window, GREEN, (x, y, player_size, player_size))

    def draw_zombie(x, y):
        pygame.draw.rect(window, RED, (x, y, zombie_size, zombie_size))

    def draw_bullet(x, y):
        pygame.draw.rect(window, WHITE, (x, y, bullet_size, bullet_size))

    def draw_collectible(x, y, type):
        color = YELLOW if type == 'trophy' else BLUE
        pygame.draw.rect(window, color, (x, y, collectible_size, collectible_size))

    def draw_stats():
        font = pygame.font.Font(None, 36)
        texts = [
            (f"Country: {country}", WHITE),
            (f"HP: {player_hp}", WHITE),
            (f"Shield: {player_shield}", BLUE),
            (f"Ammo: {ammo}", WHITE),
            (f"Food: {food}", GREEN),
            (f"Trophies: {trophies}", YELLOW),
            (f"Zombies Left: {len(zombies)}", RED)
        ]
        
        for i, (text, color) in enumerate(texts):
            rendered = font.render(text, True, color)
            window.blit(rendered, (10, 10 + i*40))

    # Game loop
    clock = pygame.time.Clock()
    running = True
    last_collectible_spawn = pygame.time.get_ticks()
    collectible_spawn_delay = 5000

    while running:
        current_time = pygame.time.get_ticks()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN and ammo > 0:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                bullet = {
                    'x': player_x + player_size // 2,
                    'y': player_y,
                    'dx': (mouse_x - (player_x + player_size // 2)) / 20,
                    'dy': (mouse_y - player_y) / 20
                }
                bullets.append(bullet)
                ammo -= 1

        # Game logic
        if current_time - last_collectible_spawn > collectible_spawn_delay:
            spawn_collectible()
            last_collectible_spawn = current_time

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and player_x > 0:
            player_x -= player_speed
        if keys[pygame.K_RIGHT] and player_x < WIDTH - player_size:
            player_x += player_speed
        if keys[pygame.K_UP] and player_y > 0:
            player_y -= player_speed
        if keys[pygame.K_DOWN] and player_y < HEIGHT - player_size:
            player_y += player_speed

        # Update bullets
        for bullet in bullets[:]:
            bullet['x'] += bullet['dx']
            bullet['y'] += bullet['dy']
            if bullet['y'] < 0 or bullet['y'] > HEIGHT or bullet['x'] < 0 or bullet['x'] > WIDTH:
                bullets.remove(bullet)

        # Update zombies and check collisions
        for zombie in zombies[:]:
            # Move zombie towards player
            if zombie['x'] < player_x:
                zombie['x'] += zombie_speed
            if zombie['x'] > player_x:
                zombie['x'] -= zombie_speed
            if zombie['y'] < player_y:
                zombie['y'] += zombie_speed
            if zombie['y'] > player_y:
                zombie['y'] -= zombie_speed

            # Update zombie rectangle for collision detection
            zombie['rect'].x = zombie['x']
            zombie['rect'].y = zombie['y']

            # Check bullet collisions
            for bullet in bullets[:]:
                if zombie['rect'].collidepoint(bullet['x'], bullet['y']):
                    zombie['health'] -= 25
                    if bullet in bullets:
                        bullets.remove(bullet)
                    if zombie['health'] <= 0 and zombie in zombies:
                        zombies.remove(zombie)
                        break

            # Check player collision
            player_rect = pygame.Rect(player_x, player_y, player_size, player_size)
            if zombie['rect'].colliderect(player_rect):
                if player_shield > 0:
                    player_shield -= 10
                else:
                    player_hp -= 10

        # Update collectibles
        for collectible in collectibles[:]:
            collectible['rect'].x = collectible['x']
            collectible['rect'].y = collectible['y']
            player_rect = pygame.Rect(player_x, player_y, player_size, player_size)
            
            if collectible['rect'].colliderect(player_rect):
                if collectible['type'] == 'ammo':
                    ammo += 20
                elif collectible['type'] == 'food':
                    food += 20
                elif collectible['type'] == 'shield':
                    player_shield = min(100, player_shield + 25)
                elif collectible['type'] == 'trophy':
                    trophies += 1
                collectibles.remove(collectible)

        # Decrease food over time
        if pygame.time.get_ticks() % 1000 == 0:
            food -= 1

        # Draw everything
        window.fill((0, 0, 0))
        draw_player(player_x, player_y)
        for zombie in zombies:
            draw_zombie(zombie['x'], zombie['y'])
        for bullet in bullets:
            draw_bullet(bullet['x'], bullet['y'])
        for collectible in collectibles:
            draw_collectible(collectible['x'], collectible['y'], collectible['type'])
        draw_stats()

        pygame.display.flip()

        # Check win/lose conditions
        if player_hp <= 0 or food <= 0:
            return False  # Lost
        if len(zombies) == 0:
            return True  # Won

        clock.tick(60)

def main():
    global current_country
    
    while True:
        current_country = 0  # Reset progress for new game
        
        for i, country in enumerate(countries):
            current_country = i
            result = show_story_screen(country)
            
            if result == "quit":
                return "quit"
            elif result == "play_again":
                return "play_again"
                
            while True:
                result = game_loop(country)
                if result:  # Won
                    break
                # If lost, retry the same country
                show_story_screen(f"You died! Retrying {country}")

# Start the game
while True:
    result = main()
    if result == "quit":
        break
    # If result is "play_again", the loop will continue

pygame.quit()
sys.exit()